<a href="https://colab.research.google.com/github/mjiii25/prac_class/blob/main/LGBM%20%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [3]:
train = pd.read_csv('/content/drive/MyDrive/energy/new_test.csv의 사본')
test = pd.read_csv('/content/drive/MyDrive/energy/new_train.csv의 사본')
submission = pd.read_csv('/content/drive/MyDrive/energy/sample_submission.csv')

In [4]:
train

,num,date_time,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,date,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
0,1,2020-08-25 00:00:00,27.800000,1.500000,74.000000,0.0,0.000000,0,0,0,25,8,2020-08-25,1,0,78.602280,0,1,0,0,0,0.000000,1.800000,30.027122,2
1,1,2020-08-25 01:00:00,27.633333,1.366667,75.333333,0.0,0.000000,0,0,1,25,8,2020-08-25,1,0,78.519273,0,1,0,0,0,0.000000,3.433333,29.859547,2
2,1,2020-08-25 02:00:00,27.466667,1.233333,76.666667,0.0,0.000000,0,0,2,25,8,2020-08-25,1,0,78.431867,0,1,0,0,0,0.000000,4.900000,29.694705,2
3,1,2020-08-25 03:00:00,27.300000,1.100000,78.000000,0.0,0.000000,0,0,3,25,8,2020-08-25,1,0,78.340060,0,1,0,0,0,0.000000,6.200000,29.533017,2
4,1,2020-08-25 04:00:00,26.900000,1.166667,79.666667,0.0,0.000000,0,0,4,25,8,2020-08-25,1,0,77.912697,0,1,0,0,0,0.000000,7.100000,29.116617,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19:00:00,28.633333,3.566667,66.000000,0.0,0.533333,1,1,19,31,8,2020-08-31,0,0,78.764020,0,1,0,0,0,2.882155,22.400000,30.894924,2
10076,60,2020-08-31 20:00:00,28.266667,3.833333,67.000000,0.0,0.266667,1,1,20,31,8,2020-08-31,0,0,78.364280,0,1,0,0,0,1.525705,25.400000,30.486546,2
10077,60,2020-08-31 21:00:00,27.900000,4.100000,68.000000,0.0,0.000000,1,1,21,31,8,2020-08-31,0,0,77.957280,0,1,0,0,0,0.000000,27.700000,30.074281,2
10078,60,2020-08-31 22:00:00,27.900000,4.100000,68.000000,0.0,0.000000,1,1,22,31,8,2020-08-31,0,0,77.957280,0,1,0,0,0,0.000000,29.300000,30.074281,2


In [5]:
test

,num,date_time,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,date,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,1,6,2020-06-01,0,0,63.43008,0,0,0,0,0,0.000000,-8.4,18.973374,1
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,1,6,2020-06-01,0,0,63.56993,0,0,0,0,0,0.000000,-16.7,18.960338,1
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,1,6,2020-06-01,0,0,63.22775,0,0,0,0,0,0.000000,-25.2,18.658598,1
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,3,1,6,2020-06-01,0,0,62.54339,0,0,0,0,0,0.000000,-34.1,18.218957,1
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,4,1,6,2020-06-01,0,0,62.39760,0,0,0,0,0,0.000000,-43.1,18.081523,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19:00:00,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,19,24,8,2020-08-24,0,0,77.80896,0,1,0,0,0,2.367647,30.8,30.000891,2
122396,60,2020-08-24 20:00:00,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,20,24,8,2020-08-24,0,0,77.44917,0,1,0,0,0,0.000000,32.3,29.525251,2
122397,60,2020-08-24 21:00:00,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,21,24,8,2020-08-24,0,0,77.44917,0,1,0,0,0,0.000000,32.5,29.487603,2
122398,60,2020-08-24 22:00:00,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,22,24,8,2020-08-24,0,0,77.52246,0,1,0,0,0,0.000000,31.3,29.276183,2


#### 군집화 결과 (수연님)

In [6]:
# 1번 군집 
list1 = [1, 31, 30, 60, 3, 5, 24,32, 33, 15]
cl1 = train[train['num'].isin(list1)==True]
print(train.shape, cl1.shape)
print('1번 군집은 전체 데이터의 {0:.2f}'.format(cl1.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1680, 25)
1번 군집은 전체 데이터의 16.67 %를 차지함


In [7]:
# 2번 군집 
list1 = [2, 6, 7, 8, 13, 14, 16, 17, 18, 22, 23, 25, 26, 27, 37, 46, 47, 48, 52, 53, 54, 55, 56, 57 ,35, 38, 43, 44, 39, 45]
cl2 = train[train['num'].isin(list1)==True]
print(train.shape, cl2.shape)
print('2번 군집은 전체 데이터의 {0:.2f}'.format(cl2.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (5040, 25)
2번 군집은 전체 데이터의 50.00 %를 차지함


In [8]:
# 3번 군집
l2 = [19,20,21,29,36,40,41,49,50,58,59]
cl3 = train[train['num'].isin(l2)==True]
print(train.shape, cl3.shape)
print('3번 군집은 전체 데이터의 {0:.2f}'.format(cl3.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1848, 25)
3번 군집은 전체 데이터의 18.33 %를 차지함


In [9]:
# 4번 군집
l3 = [14,9,10,11,12,30,42]
cl4 = train[train['num'].isin(l3)==True]
print(train.shape, cl4.shape)
print('4번 군집은 전체 데이터의 {0:.2f}'.format(cl4.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1176, 25)
4번 군집은 전체 데이터의 11.67 %를 차지함


In [10]:
# 5번 군집
l4 = [5,21,28,32,51, 34]
cl5 = train[train['num'].isin(l4)==True]
print(train.shape, cl5.shape)
print('5번 군집은 전체 데이터의 {0:.2f}'.format(cl5.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1008, 25)
5번 군집은 전체 데이터의 10.00 %를 차지함


In [11]:
# 군집 5개 완성!
cl1.reset_index(inplace=True, drop=True)
cl2.reset_index(inplace=True, drop=True)
cl3.reset_index(inplace=True, drop=True)
cl4.reset_index(inplace=True, drop=True)
cl5.reset_index(inplace=True, drop=True)

In [12]:
# 기술통계량 확인
train.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.00000,10080.000000,10080.0,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.0
mean,30.500000,27.805813,2.440585,81.962500,2.191845,0.612917,0.683333,0.483333,11.50000,28.000000,8.0,3.000000,0.285714,79.447959,0.030853,0.579167,0.302679,0.079464,0.007837,1.049931,21.134927,30.073108,2.0
std,17.318961,2.275416,1.785098,11.467641,5.704649,0.867991,0.465199,0.499747,6.92253,2.000099,0.0,2.000099,0.451776,2.411122,0.172928,0.493717,0.459440,0.270475,0.088185,3.267854,21.159186,2.402481,0.0
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.00000,25.000000,8.0,0.000000,0.000000,71.780000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.600000,24.521506,2.0
25%,15.750000,26.133333,1.200000,74.666667,0.000000,0.000000,0.000000,0.000000,5.75000,26.000000,8.0,1.000000,0.000000,77.881755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.300000,28.329378,2.0
50%,30.500000,27.466667,2.100000,83.000000,0.000000,0.100000,1.000000,0.000000,11.50000,28.000000,8.0,3.000000,0.000000,79.214220,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,17.516667,29.646428,2.0
75%,45.250000,29.100000,3.175000,91.000000,1.500000,1.033333,1.000000,1.000000,17.25000,30.000000,8.0,5.000000,1.000000,81.019620,0.000000,1.000000,1.000000,0.000000,0.000000,0.000230,33.450000,31.386323,2.0
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.000000,1.000000,23.00000,31.000000,8.0,6.000000,1.000000,85.551900,1.000000,1.000000,1.000000,1.000000,1.000000,30.285714,88.600000,38.282359,2.0


In [13]:
cl1.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.0,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.00000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.0
mean,23.400000,27.883095,2.973750,83.263095,1.869286,0.632321,0.800000,0.400000,11.500000,28.000000,8.0,3.000000,0.285714,79.748550,0.016071,0.562500,0.31369,0.095833,0.011905,1.240383,21.824544,30.125565,2.0
std,17.135191,2.228436,2.499206,11.672996,4.729929,0.898151,0.400119,0.490044,6.924248,2.000596,0.0,2.000596,0.451888,2.329356,0.125788,0.496226,0.46413,0.294450,0.108490,3.967847,21.159369,2.397801,0.0
min,1.000000,23.900000,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,8.0,0.000000,0.000000,73.364340,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-18.400000,25.767393,2.0
25%,5.000000,26.200000,1.366667,76.000000,0.000000,0.000000,1.000000,0.000000,5.750000,26.000000,8.0,1.000000,0.000000,78.172053,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,5.300000,28.326416,2.0
50%,27.000000,27.500000,2.400000,84.666667,0.000000,0.100000,1.000000,0.000000,11.500000,28.000000,8.0,3.000000,0.000000,79.456133,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,17.416667,29.673620,2.0
75%,32.000000,29.166667,3.766667,92.666667,1.100000,1.066667,1.000000,1.000000,17.250000,30.000000,8.0,5.000000,1.000000,81.158905,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,33.625000,31.465632,2.0
max,60.000000,34.600000,22.500000,100.000000,38.600000,3.000000,1.000000,1.000000,23.000000,31.000000,8.0,6.000000,1.000000,85.500240,1.000000,1.000000,1.00000,1.000000,1.000000,30.285714,80.100000,37.332811,2.0


In [14]:
cl2.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.0,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5040.0
mean,32.433333,27.883254,2.188552,81.733333,2.358571,0.624881,0.766667,0.533333,11.500000,28.000000,8.0,3.000000,0.285714,79.528602,0.027579,0.572817,0.306349,0.085317,0.007937,0.995625,22.016759,30.169107,2.0
std,17.040242,2.322270,1.568339,11.797028,6.331193,0.865704,0.422995,0.498937,6.922873,2.000198,0.0,2.000198,0.451799,2.416321,0.163781,0.494718,0.461022,0.279381,0.088742,2.834334,21.401624,2.434872,0.0
min,2.000000,22.500000,0.000000,45.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,8.0,0.000000,0.000000,72.276300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.600000,25.178917,2.0
25%,17.000000,26.158333,1.000000,74.000000,0.000000,0.000000,1.000000,0.000000,5.750000,26.000000,8.0,1.000000,0.000000,77.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.200000,28.355172,2.0
50%,36.000000,27.500000,1.800000,83.000000,0.000000,0.100000,1.000000,1.000000,11.500000,28.000000,8.0,3.000000,0.000000,79.302750,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,18.633333,29.737835,2.0
75%,47.000000,29.233333,2.966667,91.333333,1.600000,1.066667,1.000000,1.000000,17.250000,30.000000,8.0,5.000000,1.000000,81.139720,0.000000,1.000000,1.000000,0.000000,0.000000,0.028883,34.600000,31.558329,2.0
max,57.000000,35.400000,9.800000,100.000000,83.500000,3.000000,1.000000,1.000000,23.000000,31.000000,8.0,6.000000,1.000000,85.551900,1.000000,1.000000,1.000000,1.000000,1.000000,23.800000,88.600000,38.246421,2.0


In [15]:
cl3.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.00000,1848.000000,1848.0,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.0
mean,38.363636,27.688149,2.498972,82.785714,2.282143,0.584253,0.363636,0.272727,11.50000,28.000000,8.0,3.000000,0.285714,79.387289,0.042749,0.556277,0.316558,0.077381,0.007035,0.659797,19.881692,29.946983,2.0
std,14.070355,2.239035,1.571174,10.326850,5.451891,0.851714,0.481176,0.445482,6.92406,2.000541,0.0,2.000541,0.451876,2.470811,0.202345,0.496957,0.465259,0.267267,0.083600,2.462969,20.762042,2.358217,0.0
min,19.000000,22.500000,0.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.00000,25.000000,8.0,0.000000,0.000000,72.420250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.600000,25.181713,2.0
25%,21.000000,26.125000,1.300000,76.333333,0.000000,0.000000,0.000000,0.000000,5.75000,26.000000,8.0,1.000000,0.000000,77.895600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.200000,28.282663,2.0
50%,40.000000,27.400000,2.300000,83.666667,0.000000,0.066667,0.000000,0.000000,11.50000,28.000000,8.0,3.000000,0.000000,79.302750,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,17.300000,29.609267,2.0
75%,50.000000,28.900000,3.266667,90.333333,1.800000,0.933333,1.000000,1.000000,17.25000,30.000000,8.0,5.000000,1.000000,80.989660,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,30.425000,31.181651,2.0
max,59.000000,34.600000,10.100000,100.000000,40.900000,3.000000,1.000000,1.000000,23.00000,31.000000,8.0,6.000000,1.000000,85.500240,1.000000,1.000000,1.000000,1.000000,1.000000,21.600000,80.100000,37.332811,2.0


In [16]:
cl4.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.0
mean,18.285714,27.502296,3.365986,83.309524,2.369388,0.680612,0.714286,0.714286,11.500000,28.000000,8.0,3.000000,0.285714,79.198911,0.028061,0.613095,0.309524,0.045068,0.004252,1.994477,17.580867,29.725131,2.0
std,11.748861,1.941143,2.800834,10.549624,6.883993,0.943640,0.451946,0.451946,6.925132,2.000851,0.0,2.000851,0.451946,2.304114,0.165218,0.487249,0.462494,0.207542,0.065094,5.005778,18.028803,2.061047,0.0
min,9.000000,22.100000,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,8.0,0.000000,0.000000,71.780000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.900000,24.521506,2.0
25%,10.000000,26.100000,1.700000,77.000000,0.000000,0.000000,0.000000,0.000000,5.750000,26.000000,8.0,1.000000,0.000000,77.591420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.425000,28.265435,2.0
50%,12.000000,27.200000,2.700000,84.333333,0.000000,0.066667,1.000000,1.000000,11.500000,28.000000,8.0,3.000000,0.000000,79.108400,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.500000,29.384820,2.0
75%,30.000000,28.800000,4.200000,91.000000,0.950000,1.200000,1.000000,1.000000,17.250000,30.000000,8.0,5.000000,1.000000,80.909533,0.000000,1.000000,1.000000,0.000000,0.000000,0.862651,30.600000,31.082260,2.0
max,42.000000,33.600000,22.500000,100.000000,65.400000,3.000000,1.000000,1.000000,23.000000,31.000000,8.0,6.000000,1.000000,85.313360,1.000000,1.000000,1.000000,1.000000,1.000000,30.285714,73.300000,36.551888,2.0


In [17]:
cl5.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.0,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.0
mean,28.500000,27.792262,2.584127,81.692460,2.177381,0.533333,0.500000,0.500000,11.500000,28.000000,8.0,3.000000,0.285714,79.419971,0.030754,0.588294,0.296627,0.077381,0.006944,0.828375,20.956581,30.021352,2.0
std,13.896339,2.175193,1.414674,10.382957,5.010360,0.822027,0.500248,0.500248,6.925623,2.000993,0.0,2.000993,0.451978,2.334901,0.172736,0.492387,0.456997,0.267328,0.083085,2.967949,20.608268,2.348077,0.0
min,5.000000,23.900000,0.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,8.0,0.000000,0.000000,73.364340,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-18.400000,25.767393,2.0
25%,21.000000,26.266667,1.466667,75.250000,0.000000,0.000000,0.000000,0.000000,5.750000,26.000000,8.0,1.000000,0.000000,78.046310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.550000,28.409913,2.0
50%,30.000000,27.466667,2.400000,82.333333,0.033333,0.033333,0.500000,0.500000,11.500000,28.000000,8.0,3.000000,0.000000,79.179352,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,16.700000,29.646858,2.0
75%,34.000000,28.900000,3.266667,89.000000,2.250000,0.800000,1.000000,1.000000,17.250000,30.000000,8.0,5.000000,1.000000,80.909440,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,31.600000,31.181651,2.0
max,51.000000,34.600000,8.100000,100.000000,38.600000,3.000000,1.000000,1.000000,23.000000,31.000000,8.0,6.000000,1.000000,85.500240,1.000000,1.000000,1.000000,1.000000,1.000000,21.600000,75.600000,37.332811,2.0


#### LGBM 모델링

##### cluster1

In [18]:
cl1_x = cl1.drop('temperature', axis = 1)
cl1_y = cl1[['temperature']]

In [19]:
test.drop('date_time', axis = 1, inplace = True)

In [21]:
cross = KFold(n_splits = 5, shuffle = True, random_state = 42)
folds = []
for train_idx, valid_idx in cross.split(cl1_x, cl1_y):
  folds.append((train_idx, valid_idx))

In [23]:
models={}
for fold in range(5):
    print(f'===================={fold+1}=======================')
    train_idx, valid_idx=folds[fold]
    X_train=cl1_x.iloc[train_idx, :]
    y_train=cl1_y.iloc[train_idx, :]
    X_valid=cl1_x.iloc[valid_idx, :]
    y_valid=cl1_y.iloc[valid_idx, :]
    
    model=LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
             early_stopping_rounds=30, verbose=100)
    models[fold]=model
    
    print(f'================================================\n\n')

====================1=======================


ValueError: ignored